In [147]:
import numpy as np

def TInvPower(A, k, x0=None, mu0=None, tol=1e-12, max_iter=50, verbose=True):
    w = A.shape[0]
    if x0 is None:
        x = smallest_sv(A)
    else:
        x = x0
    if mu0 is None:
        mu = x.T @ A @ x
    else:
        mu = mu0
    if verbose:
        print("non-sparse mu: ", mu)
        print("non-sparse x: ", x)
    update_size = np.inf 
    it = 0
    while update_size>tol and it<max_iter:
        x /= np.linalg.norm(x)
        try:
            y = np.linalg.solve(A - mu * np.eye(w), x) # Rayleigh iteration
            # y = np.linalg.solve(A, x) # pure inverse iteration
        except:
            print("Exiting early due to singular matrix")
            return x, mu
        inds = np.argpartition(np.abs(y), -k)[-k:] # indices of largest absolute entries
        y = keep_inds(y, inds)
        y /= np.linalg.norm(y)
        #mu = y.T @ A @ y # comment out to fix mu
        update_size = min(np.linalg.norm(y - x), np.linalg.norm(y + x))/np.linalg.norm(y)
        if verbose:
            print("x:", x, "y:", y, "mu:", mu, "update_size:", update_size)
        x = y
        it += 1
    return x, mu, it

def keep_inds(vector, inds): # set all but inds of vector to 0
    temp = vector*0
    temp[inds] = vector[inds]
    return temp

def smallest_sv(A):
    U, Sigma, V = np.linalg.svd(A, full_matrices=True)
    V = V.transpose()  # since numpy SVD returns the transpose
    return V[:, -1] # smallest singular vector

In [148]:
def generate_matrix(eigs, k, noise=0): # note: first eig should be smallest one
    w = len(eigs)
    A = np.zeros((w, w))
    u_list = []
    for i in range(w):
        u_list.append(np.random.normal(0, 1, w))
    u_list[0][k:] = 0
    U = np.vstack(u_list).T
    Q, R = np.linalg.qr(U) # orthogonalize u_list
    for i in range(w):
        A += eigs[i] * np.outer(Q[:, i], Q[:, i])
    A += np.random.normal(0, noise, (w, w))
    return A, Q

np.random.seed(1) # fix random seed for reproducibility

eigs = [0.01, 0.02, 0.03, 1, 2, 3] + [0.011]*5
k = 3
A, Q = generate_matrix(eigs, k, noise=2e-5)
x, mu, it = TInvPower(A, k, mu0=0, verbose=False)
#print("A:", A)
#print("Q:", Q)
U, Sigma, V = np.linalg.svd(A, full_matrices=True)
print("Singular values of A:", Sigma)
print(f"Ended in {it} iterations")
print("x:", x, "mu:", mu, "error:", min(np.linalg.norm(x-Q[:, 0]), np.linalg.norm(x+Q[:, 0])))
print("True x:", Q[:, 0])
print("True Rayleigh quotient after noise:", Q[:, 0].T @ A @ Q[:, 0]) 

Singular values of A: [3.00000046 1.99999815 1.00000761 0.02997774 0.02002224 0.01104951
 0.01100328 0.01097603 0.01097299 0.01094947 0.0099752 ]
Ended in 50 iterations
x: [-0.8978517   0.32185157  0.30045614 -0.         -0.         -0.
  0.          0.          0.          0.         -0.        ] mu: 0 error: 0.01813513867878563
True x: [-0.89529806  0.33718466  0.29111491 -0.         -0.         -0.
 -0.         -0.         -0.         -0.         -0.        ]
True Rayleigh quotient after noise: 0.009977224978423248
